<div class="alert alert-block alert-info" style="font-size:20px">
    <b>We will be writing our own R code</b><br>
    <ul>
        <li>Upload the <i>3. JOIN_diabetes_ptsd_cohorts.ipynb</i> file from <a href="https://github.com/CWML/All-of-Us-Training-Resources/blob/main/Session%202/3.%20JOIN_diabetes_ptsd_cohorts.ipynb"><b>GitHub</b></a> to access code chunks and notes
            <ul>
                <li>File -&gt; Open -&gt; Upload</li>
                <li>Then navigate to where you saved the files for this class and choose the <i>JOIN...</i> file</li>
            </ul>
        </li>
    </ul>
</div>





# Notebook File Overview

In this notebook, we will be importing the data frames we created in the first two notebooks:
1. diabetes_control_ptsd.csv
2. diabetes_study_ptsd.csv

We will use the code snippet provided by *All of Us* to import our data frames from the workspace bucket (looks similar to the snippet used to save our data frames to the workspace bucket). 

We will do some data processing and cleaning on our existing data frames including:
* Create a new variable to designate which participants **have** and **do not have** diabetes 
* Combine the two dataframes into one dataframe
* Calculate participant age based on the current date and their DOB and create a new variable called *age*
* Select the final columns we want for our final dataframe we will analyze

# Add the code snippet from the *All of Us R and Cloud Storage snippets*

**Step 1: Run *Setup***

In [ ]:
library(tidyverse)  # Data wrangling packages.

**Step 2: Run the *copy_file_from_workspace_bucket.R* code snippet**

This will import **diabetes_control_ptsd.csv** and **diabetes_study_ptsd.csv** as data frames

**NOTE: You will have to add another line of code to each step to get both files imported**

<div style=" background-color:#f8d7da; color:#842029; border:1px solid #f5c2c7; border-radius:8px; padding:20px; font-size:20px; "> 
<b>We are going to update the provided snippet, even past the <i>DON'T CHANGE FROM HERE</i> Warning</b><br> 

<p><b>Above the <i>DON'T CHANGE FROM HERE</i> Warning</b></p> 
<ul> 
<li>Replace the single file variable: <i>name_of_file_in_bucket <- 'test.csv'</i> with two new file variables 
<ul> 
<li><i>diabetes_control_ptsd <- 'diabetes_control_ptsd.csv'</i></li> 
<li><i>diabetes_study_ptsd <- 'diabetes_study_ptsd.csv'</i></li> 
</ul> 
</li> 
</ul> 

<p><b>Below the <i>DON'T CHANGE FROM HERE</i> Warning</b></p> 
<ul style="margin-top: 8px;"> 
<li>Replace the single gsutil command: <i>system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)</i></li> 
<ul> 
<li><i>system(paste0("gsutil cp ", my_bucket, "/data/", diabetes_control_ptsd, " ."), intern=T)</i></li> 
<li><i>system(paste0("gsutil cp ", my_bucket, "/data/", diabetes_study_ptsd, " ."), intern=T)</i></li> 
</ul> 

<li style="margin-top: 15px;">Replace the single dataframe: <i>read_csv(name_of_file_in_bucket)</i></li> 
<ul> 
<li><i>diabetes_control_ptsd <- read_csv(diabetes_control_ptsd)</i></li> 
<li><i>diabetes_study_ptsd <- read_csv(diabetes_study_ptsd)</i></li> 
</ul> 

<li style="margin-top: 15px;">Replace the single head(): <i>head(my_dataframe)</i></li> 
<ul> 
<li><i>head(diabetes_control_ptsd)</i></li> 
<li><i>head(diabetes_study_ptsd)</i></li> 
</ul> 
</ul> 
</div>

In [ ]:
# This snippet assumes that you run setup first

# This code copies a file from your Google Bucket into a dataframe

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)

## CHANGE ORIGINAL TO THIS 
## ORIGINAL: name_of_file_in_bucket <- 'test.csv'
diabetes_control_ptsd <- 'diabetes_control_ptsd.csv'
diabetes_study_ptsd <- 'diabetes_study_ptsd.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket

## CHANGE ORIGINAL TO THIS 
## ORIGINAL: system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)
system(paste0("gsutil cp ", my_bucket, "/data/", diabetes_control_ptsd, " ."), intern=T)
system(paste0("gsutil cp ", my_bucket, "/data/", diabetes_study_ptsd, " ."), intern=T)

# Load the file into a dataframe

## CHANGE ORIGINAL TO THIS 
## ORIGINAL: my_dataframe  <- read_csv(name_of_file_in_bucket)
diabetes_control_ptsd  <- read_csv(diabetes_control_ptsd)
diabetes_study_ptsd  <- read_csv(diabetes_study_ptsd)


## CHANGE ORIGINAL TO THIS 
## ORIGINAL: head(my_dataframe)
head(diabetes_control_ptsd)
head(diabetes_study_ptsd)

# Clean and join the two dataframes

**Step 1: Add new columns to each data frame called *diabetes* and include values based on diabetes status**

We will use the mutate() function from the **dplyr** package to create new variables for each data frame.

In [ ]:
library(dplyr)

diabetes_no_ptsd <- diabetes_control_ptsd %>%
  mutate(diabetes = "No")

diabetes_yes_ptsd <- diabetes_study_ptsd%>%
  mutate(diabetes = "Yes")

dim(diabetes_no_ptsd)
head(diabetes_no_ptsd)
dim(diabetes_yes_ptsd)
head(diabetes_yes_ptsd)

**Step 2: Combine data frames and finalize dataset**

We will use several functions from the **dplyr** package to create our final analysis-ready dataset:

* Use the *bind_rows()* function to combine the diabetes_no_ptsd and diabetes_yes_ptsd data frames into one dataset
* Use the *rename()* function to give shorter, more manageable names to the long PTSD survey question columns
* Use the *mutate()* function to calculate participant age from their date of birth and create a new variable called *age*
* Use the *select()* function to reorder columns and remove the original date_of_birth column since we now have the calculated age


We use bind_rows() instead of a join function because the column names for each data frame are the same and in the same order, so we can simply stack the data together which is easier, cleaner, and has fewer potential errors

In [ ]:
final_diabetes_ptsd <- bind_rows(diabetes_no_ptsd, diabetes_yes_ptsd) %>%
  rename(
    ptsd_doctor = `Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?`,
    ptsd_treatment = `Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?`
  ) %>%
  mutate(
    age = floor(interval(ymd_hms(date_of_birth), today()) / years(1))
  ) %>%
select(person_id, age, everything(), -date_of_birth)



dim(final_diabetes_ptsd)
head(final_diabetes_ptsd)

# Add the code snippet from the *All of Us R and Cloud Storage snippets*

<div class="alert alert-block alert-info" style="font-size:16px">
    
<b>Navigate to the code snippets</b>
    <ul style="margin-top: 0px;">    
        <li>All of Us R and Cloud Storage snippets -> Copy file to or from Workspace Bucket -> copy_data_to_workspace_bucket.R</li>
    </ul>
    
<b>Read the code carefully</b>
    <ul style="margin-top: 0px;">    
        <li>Replace <b>df</b> with <i>THE NAME OF YOUR DATAFRAME</i>
            <ul>
                <li><i>final_diabetes_ptsd</i></li>
            </ul>
        </li>
        <li>Replace <b>'test.csv'</b> with <i>THE NAME of the file</i> you're going to store in the bucket (don't delete the quotation marks)
            <ul>
                <li><i>'final_diabetes_ptsd.csv'</i></li>
            </ul>
        </li>
    </ul>
        
</div>

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- final_diabetes_ptsd

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'final_diabetes_ptsd.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_excel_csv(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.csv"), intern=T)
